Попередня підготовка даних для роботи з ними.

In [47]:
import pandas as pd 

Імпортуємо датасет (за останні 40 років).

In [48]:
def getDataset(path):
    return pd.read_csv(path, sep=',',encoding='cp1252')

dataset = getDataset('data/medals.csv')
dataset = dataset[dataset["Year"]>=1980]

dataset.info()
dataset.head(5)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 157417 entries, 0 to 271115
Data columns (total 15 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   ID      157417 non-null  int64  
 1   Name    157417 non-null  object 
 2   Sex     157417 non-null  object 
 3   Age     156845 non-null  float64
 4   Height  149270 non-null  float64
 5   Weight  148827 non-null  float64
 6   Team    157417 non-null  object 
 7   NOC     157417 non-null  object 
 8   Games   157417 non-null  object 
 9   Year    157417 non-null  int64  
 10  Season  157417 non-null  object 
 11  City    157417 non-null  object 
 12  Sport   157417 non-null  object 
 13  Event   157417 non-null  object 
 14  Medal   21926 non-null   object 
dtypes: float64(3), int64(2), object(10)
memory usage: 13.2+ MB


,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
0,1,A Dijiang,M,24.0,180.0,80.0,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,NaN
1,2,A Lamusi,M,23.0,170.0,60.0,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,NaN
4,5,Christine Jacoba Aaftink,F,21.0,185.0,82.0,Netherlands,NED,1988 Winter,1988,Winter,Calgary,Speed Skating,Speed Skating Women's 500 metres,NaN
5,5,Christine Jacoba Aaftink,F,21.0,185.0,82.0,Netherlands,NED,1988 Winter,1988,Winter,Calgary,Speed Skating,"Speed Skating Women's 1,000 metres",NaN
6,5,Christine Jacoba Aaftink,F,25.0,185.0,82.0,Netherlands,NED,1992 Winter,1992,Winter,Albertville,Speed Skating,Speed Skating Women's 500 metres,NaN


Видалимо зайві стовпчики. Перетворимо типи даних деяких стопчиків та видалимо рядки, де немає відповідних даних.

In [49]:
def drop_columns(dataset, columns):
    for column in columns:
        dataset = dataset.drop(column, axis = 1)
    return dataset

def remove_null_rows_by_columns(dataset, columns):
    for column in columns:
        dataset = dataset[~dataset[column].isnull()]
    return dataset

def toInt(dataset,columns):
    for column in columns:
        dataset[column] = dataset[column].astype(int)

remove_columns = ["Games", "City", "Event", "Name"]
to_int_columns = ["Age", "Height", "Weight"]

dataset = drop_columns(dataset, remove_columns)
dataset = remove_null_rows_by_columns(dataset, to_int_columns)
toInt(dataset, to_int_columns)

dataset.info()
dataset.head(5)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 148083 entries, 0 to 271115
Data columns (total 11 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   ID      148083 non-null  int64 
 1   Sex     148083 non-null  object
 2   Age     148083 non-null  int32 
 3   Height  148083 non-null  int32 
 4   Weight  148083 non-null  int32 
 5   Team    148083 non-null  object
 6   NOC     148083 non-null  object
 7   Year    148083 non-null  int64 
 8   Season  148083 non-null  object
 9   Sport   148083 non-null  object
 10  Medal   21439 non-null   object
dtypes: int32(3), int64(2), object(6)
memory usage: 8.5+ MB


,ID,Sex,Age,Height,Weight,Team,NOC,Year,Season,Sport,Medal
0,1,M,24,180,80,China,CHN,1992,Summer,Basketball,NaN
1,2,M,23,170,60,China,CHN,2012,Summer,Judo,NaN
4,5,F,21,185,82,Netherlands,NED,1988,Winter,Speed Skating,NaN
5,5,F,21,185,82,Netherlands,NED,1988,Winter,Speed Skating,NaN
6,5,F,25,185,82,Netherlands,NED,1992,Winter,Speed Skating,NaN


Очистка назв країн від цифр.

In [50]:
dataset.replace('-\d*','',regex=True, inplace = True)
dataset[dataset["ID"] == 705]

,ID,Sex,Age,Height,Weight,Team,NOC,Year,Season,Sport,Medal
1253,705,M,22,190,99,Switzerland,SUI,1988,Winter,Bobsleigh,NaN
1254,705,M,26,190,99,Switzerland,SUI,1992,Winter,Bobsleigh,Gold
1255,705,M,26,190,99,Switzerland,SUI,1992,Winter,Bobsleigh,Bronze
1256,705,M,28,190,99,Switzerland,SUI,1994,Winter,Bobsleigh,Gold
1257,705,M,28,190,99,Switzerland,SUI,1994,Winter,Bobsleigh,Silver


Завантажуємо та виправляємо дані про ВВП країн.

In [51]:
GDP = getDataset('data/GDP.csv')

GDP_c = GDP.iloc[: , :2]
GDP_v = GDP.iloc[:,-42:-5]

for i in range(len(GDP_v)) :
    GDP_v.iloc[i] = GDP_v.iloc[i].fillna(value =  GDP_v.iloc[i].mean())

GDP = pd.concat([GDP_c, GDP_v], axis=1)
GDP = remove_null_rows_by_columns(GDP, ["2016"])
GDP = GDP.melt(id_vars=["Country Name", "Code"], var_name="Year", value_name="GDP")
toInt(GDP, ["Year"])

GDP.info()
GDP.head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9657 entries, 0 to 9656
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Country Name  9657 non-null   object 
 1   Code          9657 non-null   object 
 2   Year          9657 non-null   int32  
 3   GDP           9657 non-null   float64
dtypes: float64(1), int32(1), object(2)
memory usage: 188.7+ KB


,Country Name,Code,Year,GDP
0,Aruba,ABW,1980,1.815112e+09
1,Africa Eastern and Southern,AFE,1980,1.565127e+11
2,Afghanistan,AFG,1980,3.641723e+09
3,Africa Western and Central,AFW,1980,1.120313e+11
4,Angola,AGO,1980,5.934074e+09


Завантажуємо та виправляємо дані про популяції країн.

In [52]:
Pop = getDataset('data/population.csv')
Pop = drop_columns(Pop, ["Indicator Name"])
Pop_c = Pop.iloc[: , :2]
Pop_v = Pop.iloc[:,-40:-3]

for i in range(len(Pop_v)) :
    Pop_v.iloc[i] = Pop_v.iloc[i].fillna(value =  Pop_v.iloc[i].mean())

Pop = pd.concat([Pop_c, Pop_v],axis=1)
Pop = Pop[~Pop['2016'].isnull()]
Pop = Pop.melt(id_vars=["Country Name", "Country Code"], var_name="Year", value_name="Population")
toInt(Pop,["Year"])

Pop.info()
Pop.head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9731 entries, 0 to 9730
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Country Name  9731 non-null   object 
 1   Country Code  9731 non-null   object 
 2   Year          9731 non-null   int32  
 3   Population    9731 non-null   float64
dtypes: float64(1), int32(1), object(2)
memory usage: 190.1+ KB


,Country Name,Country Code,Year,Population
0,Aruba,ABW,1980,60096.0
1,Afghanistan,AFG,1980,13356511.0
2,Angola,AGO,1980,8341289.0
3,Albania,ALB,1980,2671997.0
4,Andorra,AND,1980,36067.0


Перейменуємо деякі коди країн для коректного з'єднання даних.

In [53]:
def renameCode(column, lst1,lst2):
    for x in range(len(lst1)):
        dataset[column].replace(lst1[x],lst2[x],inplace = True)  

lst1 = ["NED","IRI","ISV","BRU","CGO","GBS","BAH","VIN","SKN","GAM","GER","BUL","GRE","NCA","ALG","KUW",\
        "LIB","MAS","RSA","LBA","SUD","KSA","INA","UAE","SRI","NGR","LAT","SUI","CRC","SLO","CRO","POR",\
        "ANG","BAN","URU","PUR","HON","MRI","SEY","MTN","NIG","PHI","NEP","MGL","MON","ASA","TOG","SAM",\
        "HAI","DEN","GUI","BIZ","PAR","BER","TAN","OMA","FIJ","VAN","GUA","ESA","MAD","CHA","CAY","BAR",\
        "BOT","ANT","ZIM","GRN","MYA","MAW","TGA","GEQ","SOL","ARU","ZAM","CAM","BHU","VIE","KOS","LES","BUR"]
lst2 = ["NLD","IRN","VIR","BRN","COG","GNB","BHS","VCT","KNA","GMB","DEU","BGR","GRC","NIC","DZA","KWT",\
        "LBN","MYS","ZAF","LBY","SDN","SAU","IDN","ARE","LKA","NGA","LVA","CHE","CRI","SVN","HRV","PRT",\
        "AGO","BGD","URY","PRI","HND","MUS","SYC","MRT","NER","PHL","NPL","MNG","MCO","ASM","TGO","WSM",\
        "HTI","DNK","GIN","BLZ","PRY","BMU","TZA","OMN","FJI","VUT","GTM","SLV","MDG","TCD","CYM","BRB",\
        "BWA","ATG","ZWE","GRD","MMR","MWI","TON","GNQ","SLB","ABW","ZMB","KHM","BTN","VNM","XKX","LSO","BFA"]

renameCode("NOC",lst1,lst2)

З'єднуємо таблиці по кодам країн.

In [54]:
dataset = pd.merge(dataset, GDP,  how='left', left_on=['NOC','Year'], right_on = ['Code','Year'])
dataset = pd.merge(dataset, Pop,  how='left', left_on=['NOC','Year'], right_on = ['Country Code','Year'])
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 148083 entries, 0 to 148082
Data columns (total 17 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   ID              148083 non-null  int64  
 1   Sex             148083 non-null  object 
 2   Age             148083 non-null  int32  
 3   Height          148083 non-null  int32  
 4   Weight          148083 non-null  int32  
 5   Team            148083 non-null  object 
 6   NOC             148083 non-null  object 
 7   Year            148083 non-null  int64  
 8   Season          148083 non-null  object 
 9   Sport           148083 non-null  object 
 10  Medal           21439 non-null   object 
 11  Country Name_x  139483 non-null  object 
 12  Code            139483 non-null  object 
 13  GDP             139483 non-null  float64
 14  Country Name_y  139977 non-null  object 
 15  Country Code    139977 non-null  object 
 16  Population      139977 non-null  float64
dtypes: float64

Видаляємо непотрібні стовпці та рядки, де немає інформації про ВВП або популяцію.

In [55]:
remove_columns = ["Country Name_x", "Country Name_y", "Country Code", "NOC"]
dataset = drop_columns(dataset, remove_columns)
dataset = remove_null_rows_by_columns(dataset, ["GDP", "Population"])
dataset.info()
dataset.head(5)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 139483 entries, 0 to 148082
Data columns (total 13 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   ID          139483 non-null  int64  
 1   Sex         139483 non-null  object 
 2   Age         139483 non-null  int32  
 3   Height      139483 non-null  int32  
 4   Weight      139483 non-null  int32  
 5   Team        139483 non-null  object 
 6   Year        139483 non-null  int64  
 7   Season      139483 non-null  object 
 8   Sport       139483 non-null  object 
 9   Medal       18968 non-null   object 
 10  Code        139483 non-null  object 
 11  GDP         139483 non-null  float64
 12  Population  139483 non-null  float64
dtypes: float64(2), int32(3), int64(2), object(6)
memory usage: 10.1+ MB


,ID,Sex,Age,Height,Weight,Team,Year,Season,Sport,Medal,Code,GDP,Population
0,1,M,24,180,80,China,1992,Summer,Basketball,NaN,CHN,4.269157e+11,1.164970e+09
1,2,M,23,170,60,China,2012,Summer,Judo,NaN,CHN,8.532230e+12,1.350695e+09
2,5,F,21,185,82,Netherlands,1988,Winter,Speed Skating,NaN,NLD,2.619105e+11,1.476009e+07
3,5,F,21,185,82,Netherlands,1988,Winter,Speed Skating,NaN,NLD,2.619105e+11,1.476009e+07
4,5,F,25,185,82,Netherlands,1992,Winter,Speed Skating,NaN,NLD,3.629629e+11,1.518417e+07


Завантажуємо фінальні дані до нового csv файлу.

In [ ]:
dataset.to_csv("FullDataset.csv", sep=',', encoding='utf-8')